### load packages

In [1]:
import json
import requests
import pandas as pd
import numpy as np

### get all characters through SWAPI

In [2]:
base_url = 'https://swapi.dev/api/people/'
r = requests.get(base_url) # make a request

characters = r.json() # get all people

In [3]:
characters['results'][0]

{'name': 'Luke Skywalker',
 'height': '172',
 'mass': '77',
 'hair_color': 'blond',
 'skin_color': 'fair',
 'eye_color': 'blue',
 'birth_year': '19BBY',
 'gender': 'male',
 'homeworld': 'http://swapi.dev/api/planets/1/',
 'films': ['http://swapi.dev/api/films/1/',
  'http://swapi.dev/api/films/2/',
  'http://swapi.dev/api/films/3/',
  'http://swapi.dev/api/films/6/'],
 'species': [],
 'vehicles': ['http://swapi.dev/api/vehicles/14/',
  'http://swapi.dev/api/vehicles/30/'],
 'starships': ['http://swapi.dev/api/starships/12/',
  'http://swapi.dev/api/starships/22/'],
 'created': '2014-12-09T13:50:51.644000Z',
 'edited': '2014-12-20T21:17:56.891000Z',
 'url': 'http://swapi.dev/api/people/1/'}

In [4]:
people = []
for i in range(len(characters['results'])):
    print(len(characters['results']), ' people are found!')
    people.append(characters['results'][i])
    
while characters['next'] is not None:
    characters = requests.get(characters['next']).json()
    print(len(characters['results']), ' people are found!')
    for i in range(len(characters['results'])):
        people.append(characters['results'][i])

10  people are found!
10  people are found!
10  people are found!
10  people are found!
10  people are found!
10  people are found!
10  people are found!
10  people are found!
10  people are found!
10  people are found!
10  people are found!
10  people are found!
10  people are found!
10  people are found!
10  people are found!
10  people are found!
10  people are found!
2  people are found!


In [5]:
len(people)

82

There are 82 characters in the Star War Universe.

In [6]:
people = pd.json_normalize(people)

In [7]:
people.sample(5)

,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,films,species,vehicles,starships,created,edited,url
5,Owen Lars,178,120,"brown, grey",light,blue,52BBY,male,http://swapi.dev/api/planets/1/,"[http://swapi.dev/api/films/1/, http://swapi.d...",[],[],[],2014-12-10T15:52:14.024000Z,2014-12-20T21:17:50.317000Z,http://swapi.dev/api/people/6/
4,Leia Organa,150,49,brown,light,brown,19BBY,female,http://swapi.dev/api/planets/2/,"[http://swapi.dev/api/films/1/, http://swapi.d...",[],[http://swapi.dev/api/vehicles/30/],[],2014-12-10T15:20:09.791000Z,2014-12-20T21:17:50.315000Z,http://swapi.dev/api/people/5/
76,Shaak Ti,178,57,none,"red, blue, white",black,unknown,female,http://swapi.dev/api/planets/58/,"[http://swapi.dev/api/films/5/, http://swapi.d...",[http://swapi.dev/api/species/35/],[],[],2014-12-20T18:44:01.103000Z,2014-12-20T21:17:50.486000Z,http://swapi.dev/api/people/78/
27,Arvel Crynyd,unknown,unknown,brown,fair,brown,unknown,male,http://swapi.dev/api/planets/28/,[http://swapi.dev/api/films/3/],[],[],[http://swapi.dev/api/starships/28/],2014-12-18T11:16:33.020000Z,2014-12-20T21:17:50.367000Z,http://swapi.dev/api/people/29/
77,Grievous,216,159,none,"brown, white","green, yellow",unknown,male,http://swapi.dev/api/planets/59/,[http://swapi.dev/api/films/6/],[http://swapi.dev/api/species/36/],[http://swapi.dev/api/vehicles/60/],[http://swapi.dev/api/starships/74/],2014-12-20T19:43:53.348000Z,2014-12-20T21:17:50.488000Z,http://swapi.dev/api/people/79/


In the codebook, the birth year of each character uses the in-universe standard of BBY or ABY - Before the Battle of Yavin or After the Battle of Yavin. The Battle of Yavin is a battle that occurs at the end of Star Wars episode IV: A New Hope. In this case, characters who were born in BBY are older than those born in ABY.

There are several characters that we don't know their birth year, so we can't determine whether they are younger or older than the characters with a birth year.

In [8]:
BBY_people = people[people.birth_year.str.contains('BBY')]
ABY_people = people[people.birth_year.str.contains('ABY')]

In [9]:
BBY_people.shape[0], ABY_people.shape[0]

(43, 0)

In [10]:
np.unique(people.birth_year)

array(['102BBY', '112BBY', '15BBY', '19BBY', '200BBY', '21BBY', '22BBY',
       '24BBY', '29BBY', '31.5BBY', '31BBY', '33BBY', '37BBY', '40BBY',
       '41.9BBY', '41BBY', '44BBY', '46BBY', '47BBY', '48BBY', '52BBY',
       '53BBY', '54BBY', '57BBY', '58BBY', '600BBY', '62BBY', '64BBY',
       '66BBY', '67BBY', '72BBY', '82BBY', '896BBY', '8BBY', '91BBY',
       '92BBY', 'unknown'], dtype=object)

There are 43 characters who were born before the battle of Yavin and there are no characters who were born after the battle of Yavin.

In [11]:
BBY_people = BBY_people.assign(birth_year = BBY_people.birth_year.str.extract('(\d+)'))
BBY_people['birth_year'] = BBY_people['birth_year'].astype('int32')

In [12]:
BBY_people.loc[BBY_people['birth_year'].idxmax()]

name                                                       Yoda
height                                                       66
mass                                                         17
hair_color                                                white
skin_color                                                green
eye_color                                                 brown
birth_year                                                  896
gender                                                     male
homeworld                      http://swapi.dev/api/planets/28/
films         [http://swapi.dev/api/films/2/, http://swapi.d...
species                       [http://swapi.dev/api/species/6/]
vehicles                                                     []
starships                                                    []
created                             2014-12-15T12:26:01.042000Z
edited                              2014-12-20T21:17:50.345000Z
url                             http://s

Yoda is the oldest one among all characters with a birth year. 

### Find the titles of all the films the oldest character appeared in

In [13]:
BBY_people.loc[BBY_people['birth_year'].idxmax()]['films']

['http://swapi.dev/api/films/2/',
 'http://swapi.dev/api/films/3/',
 'http://swapi.dev/api/films/4/',
 'http://swapi.dev/api/films/5/',
 'http://swapi.dev/api/films/6/']

In [14]:
[requests.get(url).json()['title'] for url in BBY_people.loc[BBY_people['birth_year'].idxmax()]['films']]

['The Empire Strikes Back',
 'Return of the Jedi',
 'The Phantom Menace',
 'Attack of the Clones',
 'Revenge of the Sith']